In [1]:
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import pandas as pd

In [3]:
url = "https://www.berlin.de/special/shopping/adressen/biosupermarkt/?trpg=1"
response = requests.get(url)
html = response.content
soup = bs(html, "lxml")

all_addresses=soup.find_all('span', class_='address')
for span in all_addresses:
    print(span.get_text(strip=True))

Alte Jakobstr. 79-80,10179 Berlin–Mitte
Schleiermacherstrr. 25,10961 Berlin–Kreuzberg
Giesebrechtstr. 4,10629 Berlin–Charlottenburg
Greifswalder Str. 89,10409 Berlin–Prenzlauer Berg
Konstanzer Str. 2,10707 Berlin–Wilmersdorf
Koppenstr. 8,10243 Berlin–Friedrichshain
Jungfernstieg 1-2,12207 Berlin–Steglitz
Hans-Sachs-Str. 4 b,12205 Berlin–Zehlendorf
Schönhauser Allee 108,10439 Berlin–Prenzlauer Berg
Bernstorffstr. 13a,13507 Berlin–Reinickendorf


In [2]:
page = 1
stores_list = []
while page != 10:
      url = f"https://www.berlin.de/special/shopping/adressen/biosupermarkt/?trpg={page}"
      response = requests.get(url)
      html = response.content
      soup = bs(html, "lxml")
      for store in soup.find_all("article", class_="block teaser place basis"):
            store_list = {}
            store_list["heading"] = store.h3.get_text(strip=True)
            store_list["address"] = store.find("span", class_="address").get_text().replace("\n", "").strip()
            stores_list.append(store_list)
      page = page + 1

In [3]:
stores_df=pd.DataFrame(stores_list)
stores_df

,heading,address
0,Alnatura - Alte Jakobstraße,"Alte Jakobstr. 79-80, 10179 Berlin–..."
1,Alnatura - Bergmannkiez,"Schleiermacherstrr. 25, 10961 Berli..."
2,Alnatura - Giesebrechtstraße,"Giesebrechtstr. 4, 10629 Berlin–Cha..."
3,Alnatura - Greifswalder Straße,"Greifswalder Str. 89, 10409 Berlin–..."
4,Alnatura - Konstanzer Straße,"Konstanzer Str. 2, 10707 Berlin–Wil..."
...,...,...
85,Lüske,"Drakestr. 50, 12205 Berlin–Steglitz"
86,Märkische Kiste,"Motzener Str. 30, 12277 Berlin–Temp..."
87,Natürlich Bio,"Kameruner Str. 12, 13351 Berlin–Wed..."
88,Ökotussi,"Großbeerenstr. 11, 10963 Berlin–Kre..."


In [4]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="sample app")

In [19]:
stores_df["loc"] = stores_df["address"].apply(geolocator.geocode)
stores_df["point"]= stores_df["loc"].apply(lambda loc: tuple(loc.point) if loc else None)
stores_df[['lat', 'lon', 'altitude']] = pd.DataFrame(stores_df['point'].to_list(), index=stores_df.index)
stores_df=stores_df.drop(['loc','altitude','point'], axis=1)
pd.set_option("display.max_rows", 90)
stores_df=stores_df[stores_df['lat'].notna()]
stores_df.index = np.arange(1, len(stores_df) + 1)
stores_df.rename(columns={'heading': 'store_name','lat':'latitude','lon':'longitude'}, inplace=True)
stores_df

,store_name,address,latitude,longitude,longitude,latitude,longitude,latitude,longitude
1,Alnatura - Alte Jakobstraße,"Alte Jakobstr. 79-80, 10179 Berlin–...",52.510195,13.408035,13.408035,52.510195,13.408035,52.510195,13.408035
2,Alnatura - Giesebrechtstraße,"Giesebrechtstr. 4, 10629 Berlin–Cha...",52.502211,13.309167,13.309167,52.502211,13.309167,52.502211,13.309167
3,Alnatura - Greifswalder Straße,"Greifswalder Str. 89, 10409 Berlin–...",52.542979,13.441684,13.441684,52.542979,13.441684,52.542979,13.441684
4,Alnatura - Konstanzer Straße,"Konstanzer Str. 2, 10707 Berlin–Wil...",52.498704,13.312117,13.312117,52.498704,13.312117,52.498704,13.312117
5,Alnatura - Koppenstraße,"Koppenstr. 8, 10243 Berlin–Friedric...",52.512505,13.434012,13.434012,52.512505,13.434012,52.512505,13.434012
6,Alnatura - Lichterfelde Ost,"Jungfernstieg 1-2, 12207 Berlin–Ste...",52.428963,13.325958,13.325958,52.428963,13.325958,52.428963,13.325958
7,Alnatura - Lichterfelde West,"Hans-Sachs-Str. 4 b, 12205 Berlin–Z...",52.443013,13.293833,13.293833,52.443013,13.293833,52.443013,13.293833
8,Alnatura - Schönhauser Allee,"Schönhauser Allee 108, 10439 Berlin...",52.551714,13.413696,13.413696,52.551714,13.413696,52.551714,13.413696
9,Alnatura - Tegel,"Bernstorffstr. 13a, 13507 Berlin–Re...",52.591408,13.286210,13.286210,52.591408,13.286210,52.591408,13.286210
10,Alnatura - Wilmersdorfer Arcaden,"Wilmersdorfer Str. 46, 10627 Berlin...",52.508835,13.304473,13.304473,52.508835,13.304473,52.508835,13.304473


In [21]:
stores_df = stores_df.loc[:,~stores_df.columns.duplicated()].copy()
stores_df
stores_df.to_csv(r'C:\Users\Kate\Downloads\stores_data.csv')